In [1]:
import serial

import time
import pandas as pd

print(serial.__file__)

C:\Users\weesh\anaconda3\lib\site-packages\serial\__init__.py


In [2]:
import numpy as np
import random
import sys
import io
from keras.models import load_model

model2 = load_model('my_model (2).h5')

path = 'nottingham.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40

Using TensorFlow backend.
C:\Users\weesh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\weesh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\weesh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\weesh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py


corpus length: 452488
total chars: 93
{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, '<': 27, '=': 28, '>': 29, '?': 30, '@': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, '\\': 59, ']': 60, '^': 61, '_': 62, 'a': 63, 'b': 64, 'c': 65, 'd': 66, 'e': 67, 'f': 68, 'g': 69, 'h': 70, 'i': 71, 'j': 72, 'k': 73, 'l': 74, 'm': 75, 'n': 76, 'o': 77, 'p': 78, 'q': 79, 'r': 80, 's': 81, 't': 82, 'u': 83, 'v': 84, 'w': 85, 'x': 86, 'y': 87, 'z': 88, '{': 89, '|': 90, '}': 91, '~': 92}


In [3]:
print()
print('----- Generating text:')


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

start_index = random.randint(0, len(text) - maxlen - 1)
# for diversity in [0.2, 0.5, 1.0, 1.2]:
#     print('----- diversity:', diversity)

#     generated = ''
#     sentence = text[start_index: start_index + maxlen]
#     generated += sentence
#     print('----- Generating with seed: "' + sentence + '"')
# #     sys.stdout.write(generated)

#     for i in range(400):
#         x_pred = np.zeros((1, maxlen, len(chars)))
#         for t, char in enumerate(sentence):
#             x_pred[0, t, char_indices[char]] = 1.

#         preds = model2.predict(x_pred, verbose=0)[0]
#         next_index = sample(preds, diversity)
#         next_char = indices_char[next_index]

#         sentence = sentence[1:] + next_char
#         #print(i, next_char)
# #         sys.stdout.write(next_char)
# #         sys.stdout.flush()
#     print()


----- Generating text:


In [19]:
ESPSerial = serial.Serial('com4',115200)


In [20]:
def abc_get_note_chunk(strin):
    index = 0
    insideQuote = False
    
    
    if len(strin)>1:
        if (strin[:2] == '|:' or strin[:2] == ':|'):
            return strin[:2], strin[2:]
        if strin[:2] == '(3':
            return strin[:2], strin[2:]
    if len(strin)>0 and strin[0] == ':':
        return strin[:1], strin[1:]
                         
    while index < len(strin) and not strin[index] in 'zabcdefgABCDEFG':
        if strin[index] == '"':
            insideQuote = True
        index+=1
    if index >= len(strin):
        return strin, ''
    endex = index+1
    
    canWaste = False
    while (endex < len(strin) and strin[endex] in ',/0123456789im#+"') or (insideQuote and canWaste) or ('dim' in strin and strin[endex]=='d'):
        if not strin[endex] in ',/0123456789dim#+"':
            canWaste = False
        if strin[endex] == '/':
            canWaste = True
        if strin[endex] == '"' and insideQuote:
            insideQuote = False
        endex +=1
    if endex >= len(strin):
        return strin, ''
    
    return strin[:endex], strin[endex:]


diversity = .2

start_index = random.randint(0, len(text) - maxlen - 1)
# sentence = 'K:C\nEDD|CDE|DCC|C3|C3|B,3|B,3|A,3|EDD|\n'
# while len(sentence) < maxlen: 
#     sentence = ' '+sentence
sentence = text[start_index
                : start_index + maxlen]

while not 'k:' in sentence.lower():
    start_index = random.randint(0, len(text) - maxlen - 1)
    sentence = text[start_index: start_index + maxlen]
generated = ''
# print('----- Generating with seed: "' + sentence + '"')
#     sys.stdout.write(generated)

bigstr = sentence
justEncounteredKey = False
hasEncounteredKey = False
length = 0
while length<400 or not justEncounteredKey:
    justEncounteredKey = False
    if hasEncounteredKey:
        length+=1
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    preds = model2.predict(x_pred, verbose=0)[0]
#     print(preds)
    next_index = sample(preds, diversity)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char
    if hasEncounteredKey:
        bigstr = bigstr + next_char
    if next_char == 'K' or next_char=='P':
        justEncounteredKey = True
        hasEncounteredKey = True
generated += sentence
# print(sentence)
print(bigstr)

bigstr = bigstr.replace('::', ':||:')

lines = bigstr.split('\n')

# ['a': 'a']
sharpkeys = ['g','d','a', 'e', 'b']
sharps = ['f', 'c', 'g', 'd', 'a']
flatkeys = ['f']
flats = ['b']


'''
TODO: include more support for more exotic keys
'''

def cleanChunk(chunk,key):
    chunk = chunk.replace(' ','').replace('\\','').replace(':','')
    if key.lower() in sharpkeys:
        for note in sharps[:sharpkeys.index(key.lower())+1]:
            chunk = chunk.replace(note, '^'+note)
            chunk = chunk.replace(note.upper(), '^'+note.upper())
    if key.lower() in flatkeys:
        for note in flats[:flatkeys.index(key.lower())+1]:
            chunk = chunk.replace(note, '_'+note)       
            chunk = chunk.replace(note.upper(), '_'+note.upper())
    return chunk

key = 'c'
                         
chunkArr = []
placeOfLastRepeat = 0
                     
vol = False
for line in lines:
    if 'k:' in line.lower():
        key = line[len(line)-1]
    if 'p:' in line.lower():
        placeOfLastRepeat = len(chunkArr)
    if '|' in line:
        while len(line)>0:
            chunk, line = abc_get_note_chunk(line)
            if chunk == '|:':
                placeOfLastRepeat = len(chunkArr)
                continue
            elif chunk == ':' or chunk == ':|':
                print("REPEAT",placeOfLastRepeat,len(chunkArr))
#                 for i in range(10):
#                     ESPSerial.write(str.encode('c'+"&"))
                for i in range(placeOfLastRepeat, len(chunkArr),1):
                    ESPSerial.write(str.encode(chunkArr[i]+"&"))
                    print("CHUNK",chunkArr[i])
                    time.sleep(.05)
                continue
                
                vol = True
            elif chunk=='(3':
                for i in range(3):
                    chunk, line=abc_get_note_chunk(line)
                    chunk = '2'+cleanChunk(chunk,key)+'/3'
                    
                    if(not key=='c'):
                        if vol:
                            chunk = chunk+'@'
                            vol=False
                        chunkArr.append(chunk)
                        ESPSerial.write(str.encode(chunk+'&'))
                        print("CHUNK",chunk)
                continue
            chunk = cleanChunk(chunk,key)
            
            if(not key=='c' and '|' in chunk):
                vol = True

            chunk = chunk.replace('|','')
            if(not key=='c' and not '"' in chunk and len(chunk)>0):
                if vol:
                    chunk = chunk+'@'
                    vol = False
                chunkArr.append(chunk)
                ESPSerial.write(str.encode(chunk+"&"))
                print("CHUNK",chunk)

                time.sleep(.05)

# time.sleep(2)
# ESPSerial.close()

C:\Users\weesh\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


:Kerrs/Eric Foxley
M:4/4
L:1/4
K:G
P:A
(2 g/2G/2e/2g/2|\
"D"f/2d/2f/2a/2 "G"g/2f/2g/2e/2|"D"f/2d/2f/2a/2 "A7"g/2e/2c/2e/2|\
"D"d/2c/2d/2B/2 A/2G/2F/2E/2|"D"D/2F/2A/2D/2 B/2D/2F/2D/2|\
"G"B/2G/2B/2d/2 g/2d/2B/2d/2|"D7"c/2d/2c/2A/2 c/2d/2c/2A/2|\
"G"B/2A/2G/2F/2 E/2D/2B|"D"AD/2F/2 DD/2F/2|"G"G/2B/2d/2B/2 "A7"e/2c/2A/2B/2|\
"D"dd d:|
P:B
f/2g/2|"D"af d3/2d/2|"D"AF Ad|"A7"c/2e/2c AF/2E/2|"D"D/2E/2F/2G/2 AB|DF/2G/2 FE|F3/2D/2 DD|\
FD-|FE A2|FA FA|
"G"G/2F/2G/2B/2 d/2B/2G/2B/2|d/2B/2G/2B/2 d/2B/2A/2B/2|\
"G"G/2B/2G/2B/2 "D"A/2F/2D/2F/2|\
"G"G/2B/2"D"d/2A/2 "G"G/2A/2B/2d/2|"C"c/2A/2B/2G/2 "D7"A/2c/2B/2A/2|\
"G"G/2A/2B/2c/2 d/2e/2d/2c/2|"G"B/2d/2e/2f/2 g/2f/2e/2d/2|\
"C"c/2d/2e/2f/2 "G"g/2f/2g/2e/2|\
"G"d/2B/2e/2d/2 Bd/2c/2|"G"B/2G/2B/2d/2 gd|"G"B/2d/2e "D"dd/2c/2|"G"B/2d/2g/2d/2 "C"c/2d/2e/2f/2|\
"G/d/2g/2f/2e/2 "A7"d/2c/2B/2A/2|"D"dd d:|
P
CHUNK (2g/2
CHUNK G/2
CHUNK e/2
CHUNK g/2
CHUNK ^f/2@
CHUNK d/2
CHUNK ^f/2
CHUNK a/2
CHUNK g/2
CHUNK ^f/2
CHUNK g/2
CHUNK e/2
CHUNK ^f/2@
CHUNK d/2
CHUNK 

In [18]:
ESPSerial.close()



In [19]:
ESPSerial = serial.Serial('com4',115200)



In [5]:

## TWINKLE TWINKLE

line = "CCG/2G/2G/2G/2AA(3GGG FF EE D/2D/2D/2D/2 C2"
key='c'
while len(line)>0:
    chunk, line = abc_get_note_chunk(line)
    if chunk == '|:':
        placeOfLastRepeat = len(chunkArr)
        continue
    elif chunk == ':' or chunk == ':|':
        print("REPEAT",placeOfLastRepeat,len(chunkArr))
#                 for i in range(10):
#                     ESPSerial.write(str.encode('c'+"&"))
        for i in range(placeOfLastRepeat, len(chunkArr),1):
            ESPSerial.write(str.encode(chunkArr[i]+"&"))
            print("CHUNK",chunkArr[i])
            time.sleep(.05)
        continue

        vol = True
    elif chunk=='(3':
        for i in range(3):
            chunk, line=abc_get_note_chunk(line)
            chunk = '2'+cleanChunk(chunk,key)+'/3'

            if(not key=='dc'):
                if vol:
                    chunk = chunk+'@'
                    vol=False
                chunkArr.append(chunk)
                ESPSerial.write(str.encode(chunk+'&'))
                print("CHUNK",chunk)
        continue
    chunk = cleanChunk(chunk,key)

    if(not key=='dc' and '|' in chunk):
        vol = True

        
    chunk = chunk.replace('|','')
    if(not key=='dc' and not '"' in chunk and len(chunk)>0):
        if vol:
            chunk = chunk+'@'
            vol = False
        chunkArr.append(chunk)
        ESPSerial.write(str.encode(chunk+"&"))
        print("CHUNK",chunk)
        
        

        time.sleep(.04)

# time.sleep(2)        

# ESPSerial.close()

NameError: name 'abc_get_note_chunk' is not defined